In [3]:
import pandas as pd, numpy as np    # Se importan las librerías necesarias.

In [4]:
# Del archvo "user_reviews.parquet" se importa sólo las columnas "item_id", "recommend", "sentiment_analysis" y "posted" de manera de hacer más eficiente la consulta.

user_reviews = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\user_reviews.parquet",columns=['item_id','recommend','sentiment_analysis','posted'])

user_reviews.head()

,item_id,recommend,sentiment_analysis,posted
0,1250,True,2,2011-11-05
1,22200,True,1,2011-07-15
2,43110,True,2,2011-04-21
3,251610,True,1,2014-06-24
4,227300,True,1,2013-09-08


In [5]:
user_reviews['posted'] = user_reviews['posted'].dt.year  # Se extrae el año de la variable "posted".

user_reviews.head()

,item_id,recommend,sentiment_analysis,posted
0,1250,True,2,2011
1,22200,True,1,2011
2,43110,True,2,2011
3,251610,True,1,2014
4,227300,True,1,2013


In [6]:
# Del archivo "steam_games" se importan sólo las variables "developer", "id" de manera de hacer más eficiente la consulta.

steam_games = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\steam_games.parquet",columns=['developer','id'])
steam_games = steam_games.rename(columns={'id':'item_id'}) # Se renombra la variable "id" a "item_id" para poder unir con el dataframe "user_reviews".
steam_games.head()

,developer,item_id
0,Kotoshiro,761140
1,Secret Level SRL,643980
2,Poolians.com,670290
3,彼岸领域,767400
4,Trickjump Games Ltd,772540


In [7]:
data = pd.merge(user_reviews,steam_games,how='inner',on='item_id') # Se unen los dos dataframes a través de la variable "item_id", con sólo los valores en común, evitando los valores nulos.

data.head()

,item_id,recommend,sentiment_analysis,posted,developer
0,1250,True,2,2011,Tripwire Interactive
1,22200,True,1,2011,ACE Team
2,227300,True,1,2013,SCS Software
3,239030,True,2,2013,3909
4,370360,True,2,2015,Zachtronics


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40086 entries, 0 to 40085
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             40086 non-null  int64 
 1   recommend           40086 non-null  bool  
 2   sentiment_analysis  40086 non-null  int64 
 3   posted              40086 non-null  int32 
 4   developer           40086 non-null  object
dtypes: bool(1), int32(1), int64(2), object(1)
memory usage: 1.1+ MB


In [9]:
data = data[(data['recommend']==True) | (data['sentiment_analysis']==2)] # Se filtran los resultados donde "recommend = Si" o "sentiment_analysis = Positivo".

data.head()

,item_id,recommend,sentiment_analysis,posted,developer
0,1250,True,2,2011,Tripwire Interactive
1,22200,True,1,2011,ACE Team
2,227300,True,1,2013,SCS Software
3,239030,True,2,2013,3909
4,370360,True,2,2015,Zachtronics


In [10]:
count_recommended_games = data.groupby(by=['posted','developer'])['item_id'].count()  # Se cuenta el número de elementos por año y por desarrollador


count_recommended_games.head()

posted  developer          
2010    1C Company             1
        1C: Maddox Games       1
        Amanita Design         1
        BioWare                1
        BioWare,Aspyr (Mac)    1
Name: item_id, dtype: int64

In [11]:
count_recommended_games.drop_duplicates(inplace=True) # Se elimian los casos duplicados.

In [12]:
best_developer_year = data.merge(count_recommended_games,on=['posted','developer'])  # El conteo previamente creado se une con el dataframe "data".

best_developer_year.head()

,item_id_x,recommend,sentiment_analysis,posted,developer,item_id_y
0,1250,True,2,2011,Tripwire Interactive,20
1,239030,True,2,2013,3909,14
2,250320,True,2,2013,Telltale Games,110
3,207610,True,1,2012,Telltale Games,17
4,211820,True,2,2013,Chucklefish,99


In [13]:
best_developer_year = best_developer_year.rename(columns={'item_id_x':'item_id','posted':'Año','item_id_y':'Conteo'}) # Se renombran las variables.

best_developer_year.head()

,item_id,recommend,sentiment_analysis,Año,developer,Conteo
0,1250,True,2,2011,Tripwire Interactive,20
1,239030,True,2,2013,3909,14
2,250320,True,2,2013,Telltale Games,110
3,207610,True,1,2012,Telltale Games,17
4,211820,True,2,2013,Chucklefish,99


In [14]:
best_developer_year = best_developer_year[['Año','developer','Conteo']] # Se seleccionan sólo las columnas necesarias.

best_developer_year.head()

,Año,developer,Conteo
0,2011,Tripwire Interactive,20
1,2013,3909,14
2,2013,Telltale Games,110
3,2012,Telltale Games,17
4,2013,Chucklefish,99


In [15]:
best_developer_year = best_developer_year.drop_duplicates()

In [16]:
best_developer_year.to_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\Apis\best_developer_year\bdy.parquet",compression='snappy')